## **Get the data via API for making the graph (server)**

In [17]:
from datetime import date, timedelta, datetime

import requests

def get_graph_data(ticker):
    """
    
        Obtains the data of the stock price evolution for a given ticker over the last year

        Parameters:
            ticker (str): Stock ticker symbol
        
        Returns:
            A dictionary containing the data for making the plot with Plotly {dates: [], prices: []}
    
    """
    
    # Variables and API setup
    API_KEY = 'ea51535a06ab42f0824812f815f2eb08' 
    OUTPUT_SIZE = 252
    URL = 'https://api.twelvedata.com/time_series'
    START_DATE = (date.today() - timedelta(days=365)).isoformat()

    params = {
        'symbol': ticker,
        'interval': '1day',
        'outputsize': OUTPUT_SIZE,
        'start_date': START_DATE,
        'order': 'asc',
        'apikey': API_KEY
    }
    
    # Make the API request
    try:
        response = requests.get(URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        values = data['values']
        
        # Extract dates and prices
        dates, prices = [], []
        for day in values:
            # Para que la fecha tenga el mismo formato que las news y poder plotear
            date_obj = datetime.strptime(day['datetime'], '%Y-%m-%d')
            dates.append(date_obj)
            prices.append(float(day['close']))
        
        return {'ticker': ticker, 'dates': dates, 'prices': prices}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None


## **Make the graph with the previous data (client)**

In [29]:
import plotly.graph_objects as go

def make_graph(graph_data, news):
    """
    Create a Plotly graph for stock price evolution and save it as an image.
    
    Parameters:
        graph_data (dict): A dictionary containing 'dates' and 'prices' lists.
    
    Returns:
        str: The file path to the saved HTML graph.
    """
    ticker, dates, prices = graph_data['ticker'], graph_data['dates'], graph_data['prices']
    
    # 1. Plotea el gráfico de la evolución del stock
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dates, 
        y=prices, 
        mode='lines', 
        name=ticker.upper(),
        line=dict(color='royalblue', width=2),
        hovertemplate='<b>Precio:</b> $%{y:.2f}<extra></extra>'
    ))

    for new in news:
        date_new = new[0]
        title_new = new[1]
        
        # Plotea solo las noticias que coinciden con dias en los que la bolsa está abierta
        if date_new in dates:
            # Plotea las noticias como lineas verticales infinitas
            fig.add_vline(
                x=date_new, 
                line_width=1, 
                line_dash="dash", 
                line_color="grey"
            )

            fig.add_trace(go.Scatter(
                x=[date_new],
                y=[prices[dates.index(date_new)]],
                mode='markers',
                marker=dict(size=0, opacity=0),
                name='Noticia',
                text=[f"<b>Noticia:</b> {title_new}"],
                hovertemplate='%{text}<extra></extra>',
                showlegend=False
            ))

    fig.update_layout(
        title = f'{ticker.upper()} Stock Price Evolution Over the Last Year',
        xaxis_title = 'Date',
        yaxis_title = 'Closing Price (USD)',
        hovermode='x unified',
        hoverdistance=5,
        template = 'plotly_white',
        hoverlabel = dict(bgcolor="white", font_size=13, font_family="Rockwell")
    )
    
    fig.write_html(f"stock_price_graph.html")

    return "stock_price_graph.html"

## **FMD API for getting the analysts opinion (server)**

In [ ]:
import requests

def get_estimations(ticker):
    """
    
        Fetch financial estimations from Financial Modeling Prep API
        
        Parameters:
            ticker: str - The stock ticker symbol to search for.
        Returns:
            Dictionary with the fetched estimations (key-value pairs including metrics like price target, earnings estimates, etc.)
    
    """
    
    # Financial Modeling Prep (FMP) API key for estimations
    API_KEY = 'm6B6VyNRoaMYJOIxJPWLzD6K9oVopgoe'
    
    # Prepare the URLs
    URL_FINANCIAL_ESTIMATES = f'https://financialmodelingprep.com/stable/analyst-estimates'
    params_financial_estimates = {
        'apikey': API_KEY,
        'symbol': ticker.upper(),
        'period': 'annual'
    }
    
    URL_PRICE_TARGET_CONSENSUS = f'https://financialmodelingprep.com/stable/price-target-consensus'
    params_price_target_consensus = {
        'apikey': API_KEY,
        'symbol': ticker.upper()
    }
    
    URL_STOCK_GRADES = f'https://financialmodelingprep.com/stable/grades-consensus'
    params_stock_grades = params_price_target_consensus
    
    # Function to fetch data
    def fetch_data(url, params):
        try:
            # Make the GET request to the API endpoint
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            # Process the response data
            if isinstance(data, list):
                return data[0] if data else {}
            
            if isinstance(data, dict):
                return data
            
            # Handle unexpected data format
            print(f"Unexpected data format from {url}: {type(data)}")
            return {}
        
        # Handle request exceptions
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching data from {url}: {e}")
            return {}
    
    # Fetch data from the three endpoints
    financial_estimates = fetch_data(URL_FINANCIAL_ESTIMATES, params_financial_estimates)
    price_target_consensus = fetch_data(URL_PRICE_TARGET_CONSENSUS, params_price_target_consensus)
    stock_grades = fetch_data(URL_STOCK_GRADES, params_stock_grades)
    
    # Return combined data
    return {**financial_estimates, **price_target_consensus, **stock_grades}


## **Finviz.com webscrape via `Selenium` for obtaing the company information (server)**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_information(ticker):
    """
    
        Selenium Web Scraping from finviz.com
        
        Parameters:
            ticker: str - The stock ticker symbol to search for.
        Returns:
            Dictionary with the extracted information (key-value pairs including metrics like P/E ratio, market cap, etc.)
    
    """
    # Hide warnings from Selenium
    options = Options()
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    # Initialize the WebDriver and information dictionary
    driver = webdriver.Chrome(options=options)
    info = {}
    
    # Full screen the window
    driver.maximize_window()
    
    try:
        # Access the website
        driver.get("https://finviz.com/")
        
        # Handle cookie consent pop-up
        time.sleep(1)  # Wait for the pop-up to appear

        """leer_mas = driver.find_element(By.CSS_SELECTOR, ".Button__StyledButton-buoy__sc-a1qza5-0.elJono")
        if leer_mas:
            leer_mas.click()"""

        cookie_reject_button = driver.find_element(By.CLASS_NAME, "Button__StyledButton-buoy__sc-a1qza5-0")
        cookie_reject_button.click()
        
        # Locate the search input field and enter the ticker symbol. Then wait 0.5 seconds and press ENTER
        search_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.TAG_NAME, "input")))
        search_input.click()
        search_input.send_keys(ticker.upper())
        time.sleep(0.5)
        search_input.send_keys(Keys.ENTER)
        
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "snapshot-table2")))
        
        # Extract the required information
        table = driver.find_element(By.CLASS_NAME, "snapshot-table2")
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Iterate through rows and extract key-value pairs
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            for i in range(0, len(cells), 2):
                key = cells[i].text
                value = cells[i + 1].text
                info[key] = value
    
    # Handle exceptions
    except Exception as e:
        print(f"An error occurred: {e}")
    
    # Ensure the driver is closed properly
    finally:
        time.sleep(1)
        driver.quit()
    
    # Return the extracted information
    return info


## **Elmundo.es webscrape via `Selenium` for getting the latest news (server)**

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import yfinance as yf
import re

def get_news(ticker):
    """
    
    Selenium Web Scraping from elmundo.es for news related to the company represented by the ticker symbol.
    
    Parameters:
        ticker: str - The stock ticker symbol to search for.
        
    Returns:
        List of tuples containing (date, title, link) of relevant news articles.
    
    """
    
    # Get company name from ticker
    try:
        empresa = yf.Ticker(ticker).info['shortName']
        empresa = re.search(r'\w+', empresa).group(0)  # Take only the first word of the company name
    except:
        empresa = ticker  # If fails, use ticker as company name
    
    # Hide warnings from Selenium
    options = Options()
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    
    # Initialize WebDriver
    try:
        driver = webdriver.Chrome(options=options)
        driver.get('https://www.elmundo.es/')
        driver.maximize_window()
    
    except:
        print("Error initializing WebDriver or accessing elmundo.es")
    
    # Handle cookies pop-up
    try:
        cookies_button=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.ID, "ue-accept-notice-button"))
                )
        cookies_button.click()
    except:
        print("Error handling cookies pop-up")
    
    # Click search button
    try:
        search_button=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "ue-c-main-header__search-box"))
                )
        search_button.click()
    except:
        print("Error clicking search button")
    
    # Click advanced search
    try:
        busqueda_avanzada=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "búsqueda avanzada »"))
                )
        busqueda_avanzada.click()
    except:
        print("Error clicking advanced search")
        
    # Choose 50 results per page
    try:
        WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'consejos'))) # Wait for all options to load
        desplegables = driver.find_elements(By.CLASS_NAME, 'consejos') # Get all options
        select = Select(desplegables[2]) # Take the third
        select.select_by_value("50")
    except: 
        print("Error selecting the number of results per page")

    # Choose news from the last year
    try:
        WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'consejos')) # Wait for all options to load
                )
        select = Select(desplegables[0]) # Take the first
        select.select_by_value("365")
    except: 
        print("Error selecting the number of results per page")
        
    # Choose 70% match percentage
    try:
        select = Select(desplegables[3]) # Take the fourth
        select.select_by_value("70")
    except: 
        print("Error selecting the match percentage")
        
    # Send keys of the company name and submit
    try:
        insertar_nombre=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.ID, "q"))
                )
        insertar_nombre.send_keys(empresa)
        insertar_nombre.send_keys(Keys.ENTER)
    
    except:
        print("Error sending keys of the company name")
        
    # Click a button to sort news by date
    try:
        boton_ordenar_fecha=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "Ordenar por FECHA"))
                )
        boton_ordenar_fecha.click()
    except:
        print("Error with the button to sort by date")

    # Click on economy to only show economic news
    try:
        boton_economia=WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, "Economía"))
                )
        boton_economia.click()
    except:
        print("Error with the button for economic news")
    
    lista=[]
    # Collect data from each page and then click on the next page button if it exists; if it doesn't exist, break the loop
    while True:
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "li"))) # Wait for all li elements to load
            
            todos_los_li = driver.find_elements(By.TAG_NAME, "li")
        
            # We only save the news that have the company name in their text (title or subtitle)
        except:
            print("Error getting the li elements of the news")
        
    # Filter important news and save dates (coherence 80% or more)
        try:
            for i in todos_los_li:
                if empresa.lower() in i.text.lower():
                    elemento_a = i.find_element(By.TAG_NAME, 'a')
                    titulo = elemento_a.text
                    enlace = elemento_a.get_attribute('href')
                    fecha_sucia = i.find_element(By.CLASS_NAME, 'fecha').text
                    fecha_object = datetime.strptime(fecha_sucia, '%d/%m/%Y')
                    lista.append((fecha_object, titulo, enlace))
        except:
            print("Error filtering that the company name is in the title or subtitle and saving it")
            
    # Go to the next page    
        try:
            boton_siguiente_pag=WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.LINK_TEXT, "Siguiente »"))
            )
            
            boton_siguiente_pag.click()
        except:
            break
    
    # Close the driver
    time.sleep(1)
    driver.quit()
    
    # Return the list of news
    return lista


## **Generate financial summary from previous information and estimations (server)**

In [ ]:
from tabulate import tabulate

def generate_financial_summary(raw_data):
    """
    
        Filter financial data and generate a formatted table with key metrics and interpretations.
        
        Parameters:
            raw_data: dict - Dictionary containing raw financial metrics from web scraping and API calls.
        Returns:
            String containing a formatted table with filtered metrics, their values, and economic interpretations.
    
    """
    
    # Definition of metrics to filter and their explanations
    # Format: 'Original_Key': ('Readable_Name', 'Description/Importance')
    metrics_map = {
        'Price': ('Current Price', 'Current market value of the stock.'),
        'Market Cap': ('Market Capitalization', 'Total size of the company in the market.'),
        'Perf Year': ('Annual Performance', 'Percentage change in stock price over the last year.'),
        'P/E': ('P/E Ratio', 'Price/Earnings. Indicates how expensive the stock is.'),
        'Forward P/E': ('Forward P/E', 'Expected Price/Earnings ratio (lower values indicate potential improvement).'),
        'Target Price': ('Target Price', 'Price that analysts expect within 12 months.'),
        'consensus': ('Consensus', 'Average analyst opinion (Buy/Hold/Sell).'),
        'targetHigh': ('Analysts Ceiling', 'The most optimistic price target recorded.'),
        'EPS next Y': ('EPS Growth', 'Expected earnings growth for the next year.'),
        'ROE': ('ROE (%)', 'Return on Equity. Measures efficiency of capital use.'),
        'Debt/Eq': ('Debt/Capital', 'Leverage level. Low values indicate financial strength.'),
        'Profit Margin': ('Profit Margin', 'Percentage of revenue converted to profit.'),
        'RSI (14)': ('RSI Index', 'Indicates if the stock is overbought (>70) or oversold (<30).'),
        'revenueAvg': ('Revenue 2030 (Est)', 'Long-term revenue projection according to API.'),
        'numAnalystsEps': ('Number of Analysts', 'Quantity of experts covering this company.')
    }

    table_data = []
    
    # Filtering process (requirement for extra points)
    for key, (label, description) in metrics_map.items():
        value = raw_data.get(key, "N/A")
        table_data.append([label, value, description])

    # Table generation with tabulate
    headers = ["Metric", "Value", "Economic Interpretation"]
    
    return tabulate(table_data, headers=headers, tablefmt='rounded_grid')


## **ERROR: Trying `BeautifulSoup & requests` to webscrape but raised 401 error (Not authorized)**

In [ ]:
def get_estimations(ticker):
    # Ensure necessary imports
    from bs4 import BeautifulSoup
    import requests
    
    url = f"https://marketwatch.com/investing/stock/{ticker.lower()}/analystestimates"
    
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
                "Accept-Language": "en-US,en;q=0.9",
                "Referrer": "https://google.com"}
    
    try:
        response = requests.get(url, headers=headers)
    
        if response.status_code != 200:
            print(f"Failed to retrieve data for {ticker.upper()}. Status code: {response.status_code}")
            return {}
    
        soup = BeautifulSoup(response.content, "html.parser")
        estimations = {}
    
        try:
            tables = soup.find_all("table", class_ = "table value-pairs no-heading font--lato")
            
            for i in range(2):
                for row in tables[i].find_all("tr"):
                    cells = row.find_all("td")
                    if len(cells) >= 2:
                        key = cells[0].get_text(strip=True)
                        value = cells[1].get_text(strip=True)
                        estimations[key] = value
            
            if estimations:
                print(f"Estimations for {ticker.upper()} extracted successfully")
            else:
                print(f"No estimations found for {ticker.upper()}")

        except Exception as e:
            print(f"An error occurred while parsing the estimations: {e}")

    except Exception as e:
        print(f"An error occurred while extracting estimations: {e}")
    
    return estimations


## **ERROR: Trying `Selenium` to webscrape but bot was detected**

In [ ]:
def get_estimations(driver, ticker):
    # Ensure necessary imports
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    # Go directly to the MarketWatch analyst estimates page for the ticker
    url = f"https://www.marketwatch.com/investing/stock/{ticker.lower()}/analystestimates"
    estimations = {}

    try:
        driver.get(url)
        
        print("Acceded to MarketWatch analyst estimates page")
        time.sleep(1) # Wait for the page to load
        
        # Cookie pop-up handling 
        try:
            # Wait until appears the iframe containing the cookie button
            iframe = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//iframe[contains(@id, 'sp_message_iframe_1396985')]")))
            
            # Switch to the iframe
            driver.switch_to.frame(iframe)
            
            # Wait for the cookie button to be clickable and click it
            cookie_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text() = 'YES, I AGREE']")))
            cookie_btn.click()
            
            print("Cookie pop-up closed.")
        
        except Exception as e:
            print("No cookie pop-up found or already closed. Exception:", e)

        # Ensure we are at the main content
        driver.switch_to.default_content()

        # Wait for the estimation tables to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "table value-pairs no-heading font--lato")))

        # Find all the estimation tables
        tables = driver.find_elements(By.CLASS_NAME, "table value-pairs no-heading font--lato")
        
        # For the first two tables, extract key-value pairs
        for i in range(2):
            rows = tables[i].find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    estimations[key] = value
        
        # If no estimations were found, log a warning
        if not estimations:
            print(f"Warning: No data scraped for {ticker}")
        else:
            print(f"Successfully scraped analyst data for {ticker}")

    except Exception as e:
        print(f"Error scraping MarketWatch for {ticker}: {e}")

    return estimations